In [ ]:
# Add the parent directory to the path
import sys, os
sys.path.insert(0, os.path.abspath("../.."))

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt

import torch
import random
from tabulate import tabulate

from src.data.config import HP

from src.solvers.spnia_asym import AsymmetricSPNI
from src.models.ShortestPathGrb import shortestPathGrb
from src.models.ShortestPathGrid import ShortestPathGrid
from src.solvers.BendersDecomposition import BendersDecomposition

from scripts.compare_po_spo import compare_po_spo
from scripts.setup import gen_test_data, gen_train_data, setup_po_model, setup_spo_model

In [ ]:
# Define hyperparameters
c_min: float = 1.0
c_max: float = 10.0
d_min: float = 1.0
d_max: float = 10.0
Q = 0.6
B = 5
network = (6, 8)
random_seed = 31

# ML hyperparameters
num_features = 5
num_data_samples = 200
test_size = 0.2
data_loader_batch_size = 32
po_epochs = 60
spo_epochs = 20
po_lr = .01
spo_lr = .003
sim_data_samples = 100 # number of training data
deg = 5
noise_width = 0.05

# Interdictor parameters
benders_max_count = 20
benders_eps = 1e-3
lsd = 1e-5

# Set the normalization parameter as a function of the degree
normalization_constant = {3:10, 5:20, 7:70}[deg]

# Set the random seed for reproducibility
np.random.seed(random_seed)
random.seed(random_seed)
torch.manual_seed(random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_seed)

# Initialize the configuration class
cfg = HP()

# Change configuration parameters (optional)
cfg.set("c_min", c_min)
cfg.set("c_max", c_max)
cfg.set("d_min", d_min)
cfg.set("d_max", d_max)
cfg.set("Q", Q)
cfg.set("budget", B)
cfg.set("grid_size", network)
cfg.set("random_seed", random_seed)

cfg.set("num_features", num_features)
cfg.set("num_data_samples", num_data_samples)
cfg.set("test_size", test_size)
cfg.set("data_loader_batch_size", data_loader_batch_size)
cfg.set("po_epochs", po_epochs)
cfg.set("spo_epochs", spo_epochs)
cfg.set("po_lr", po_lr)
cfg.set("spo_lr", spo_lr)
cfg.set("sim_data_samples", sim_data_samples)
cfg.set("deg", deg)
cfg.set("noise_width", noise_width)
cfg.set("normalization_constant", normalization_constant)

cfg.set("benders_max_count", benders_max_count)
cfg.set("benders_eps", benders_eps)
cfg.set("lsd", lsd)


In [ ]:
# Define a graph with appropriate dimensions and an opt_model 
# for solving the shortest path problem on the graph
m,n = cfg.get("grid_size")
graph = ShortestPathGrid(m,n)
opt_model = shortestPathGrb(graph)

# Generate training data
training_data = gen_train_data(cfg, opt_model)

In [ ]:
po_model = setup_po_model(
    cfg,
    graph,
    opt_model,
    training_data,
    versatile=True
)

In [ ]:
cfg.set("spo_lr", 0.001)
spo_model = setup_spo_model(
    cfg,
    graph,
    opt_model,
    training_data,
    versatile=True,
    transfer_model=po_model
)

In [ ]:
test_data = gen_test_data(cfg)
true_objs, po_objs, spo_objs = compare_po_spo(cfg, opt_model, po_model, spo_model, test_data)

In [ ]:
po_val = [(po - true) / true * 100 for po, true in zip(po_objs, true_objs)]
spo_val = [(spo - true) / true * 100 for spo, true in zip(spo_objs, true_objs)]

# plt.scatter(np.ones_like(po_objs), po_val)
# plt.scatter(np.ones_like(spo_objs) + 1, spo_val)

fig, ax = plt.subplots(figsize=(6,4))

# Two groups: PO and SPO
_ = ax.boxplot([po_val, spo_val], tick_labels=['PO', 'SPO'],
           showmeans=True, meanline=True)   # optional styling
ax.set_ylabel("Cost increase [%]")
# plt.legend(['PO', 'SPO'], location='north')

po_mean  = np.mean(po_val)
spo_mean = np.mean(spo_val)

# std devs
po_std_pop  = np.std(po_val, ddof=0)   # population σ
po_std_samp = np.std(po_val, ddof=1)   # sample s (n-1)
spo_std_pop  = np.std(spo_val, ddof=0)
spo_std_samp = np.std(spo_val, ddof=1)
mean_improvement = (po_mean - spo_mean) * 100 / po_mean
std_improvement = (po_std_samp - spo_std_samp) * 100 / po_std_samp

rows = [
    ["Mean",         f"{po_mean:.2f}%",  f"{spo_mean:.2f}%", f"{mean_improvement:.2f}%"],
    ["Std (sample)", f"{po_std_samp:.2f}%", f"{spo_std_samp:.2f}%", f"{std_improvement:.2f}%"],
]
print(tabulate(rows, headers=["Metric", "PO (%)", "SPO (%)", "SPO improvement (%)"], tablefmt="github"))

In [ ]:
interdictions = gen_test_data(cfg)

In [ ]:
def simulate_interdictor(cfg, test_data, interdictions, est_model = None, idx = None):
    """
    Compare the performance of the PO and SPO models using symmetric shortest path interdiction.
    This function simulates the interdiction process and evaluates the objective values.
    """

    # Get the number of simulation data samples
    sim_data_samples = cfg.get("sim_data_samples")
    normalization_constant = cfg.get("normalization_constant")

    # Prepare lists to store results
    true_objs = []
    est_objs = []

    # Print that the simulation is starting
    print(f"Running simulation with {sim_data_samples} samples...")

    # Iterate through each data sample
    for i in range(sim_data_samples) if idx is None else [idx]:
        # Store values for the current sample
        cost = test_data["costs"][i] * normalization_constant
        interdiction = interdictions["costs"][i] * normalization_constant
        m, n = cfg.get("grid_size")

        # Compute estimated cost if an estimator is provided. Otherwise use the true costs
        if est_model is not None:
            feature = test_data["features"][i]
            est_cost = est_model(torch.tensor(feature, dtype=torch.float32)).detach().numpy() * normalization_constant
        else:
            est_cost = cost

        # Update opt_model
        est_graph = ShortestPathGrid(m, n, cost=est_cost)
        est_opt_model = shortestPathGrb(est_graph)

        # Solutions without information asymmetry
        interdictor_I = BendersDecomposition(est_opt_model, 
                                             k=cfg.get("budget"), 
                                             interdiction_cost=interdiction, 
                                             max_cnt=cfg.get("benders_max_count"), 
                                             eps=cfg.get("benders_eps"))
        x_intd, y_est, _ = interdictor_I.solve(versatile=False if idx is None else True)

        # Create true model
        true_graph = ShortestPathGrid(m, n, cost=cost)
        true_opt_model = shortestPathGrb(true_graph)

        # True shortest path after interdiction
        true_opt_model.setObj(cost + x_intd * interdiction)
        y_true, _ = true_opt_model.solve()

        # Store the results
        true_objs.append(true_graph(y_true, interdictions=x_intd * interdiction))
        est_objs.append(true_graph(y_est, interdictions=x_intd * interdiction))

        # Print progress
        step = max(1, sim_data_samples // 20)  # ~every 5% (safe for small N)
        if (i % step == 0) or (i == sim_data_samples - 1):
            if i == sim_data_samples - 1:
                done, pct = 20, 100                      # final tick
            else:
                done = (i * 20) // sim_data_samples      # 0..20 “=”
                pct  = done * 5                          # 0,5,10,...,95

            sys.stdout.write('\r[%-20s] %3d%%' % ('=' * done, pct))
            sys.stdout.flush()
            if i == sim_data_samples - 1:
                sys.stdout.write('\n')


    # Evaluate performance
    return {
        "true_objective": np.array(true_objs),
        "est_objective": np.array(est_objs),
    }

# Debugging Bender's Decomposition!

In [ ]:
cost = test_data["costs"][60] * normalization_constant
interdiction = interdictions["costs"][60] * normalization_constant
m, n = cfg.get("grid_size")
est_cost = cost

# Update opt_model
est_graph = ShortestPathGrid(m, n, cost=est_cost)
est_opt_model = shortestPathGrb(est_graph)
est_opt_model.visualize()

print(cost)

In [ ]:
# Solutions without information asymmetry
# cfg.set("benders_max_count", 24)
interdictor_I = BendersDecomposition(est_opt_model, 
                                        k=cfg.get("budget"), 
                                        interdiction_cost=interdiction, 
                                        max_cnt=cfg.get("benders_max_count"), 
                                        eps=cfg.get("benders_eps"))
x_intd, y_est, _ = interdictor_I.solve(versatile=True, visualize=True)

In [ ]:
true_graph = ShortestPathGrid(m, n, cost=cost+x_intd * interdiction)
true_opt_model = shortestPathGrb(true_graph)
y_true, obj = true_opt_model.solve(visualize=True)

In [ ]:
cfg.set("benders_max_count", 20)

In [ ]:
idx = 0

In [ ]:
true_interdiction = simulate_interdictor(cfg, test_data, interdictions)

In [ ]:
po_interdiction = simulate_interdictor(cfg, test_data, interdictions, est_model=po_model)

In [ ]:
spo_interdiction = simulate_interdictor(cfg, test_data, interdictions, est_model=spo_model)

In [ ]:
print(true_interdiction['true_objective'].mean())
print(true_interdiction['est_objective'].mean())

In [ ]:
print(po_interdiction['true_objective'].mean())
print(po_interdiction['est_objective'].mean())

In [ ]:
print(spo_interdiction['true_objective'].mean())
print(spo_interdiction['est_objective'].mean())

In [ ]:
print(true_interdiction['true_objective'][60])
print(true_interdiction['est_objective'][60])
print(po_interdiction['true_objective'][60])
print(spo_interdiction['true_objective'][60])

In [ ]:
idx = 0
for po , true in zip(po_interdiction['true_objective'], true_interdiction['true_objective']):
    if true < po:
        print(idx)
    idx += 1

In [ ]:
po_vals = [(true - po) / true * 100 for po, true in zip(po_interdiction['true_objective'], true_interdiction['true_objective'])]
spo_vals = [(true - spo) / true * 100 for spo, true in zip(spo_interdiction['true_objective'], true_interdiction['true_objective'])]

po_mean  = np.mean(po_vals)
spo_mean = np.mean(spo_vals)

# # std devs
# po_std_pop  = np.std(po_val, ddof=0)   # population σ
# po_std_samp = np.std(po_val, ddof=1)   # sample s (n-1)
# spo_std_pop  = np.std(spo_val, ddof=0)
# spo_std_samp = np.std(spo_val, ddof=1)
# mean_improvement = (po_mean - spo_mean) * 100 / po_mean
# std_improvement = (po_std_samp - spo_std_samp) * 100 / po_std_samp

# Create figure
fig, ax = plt.subplots(figsize=(6,4))

# Two groups: PO and SPO
_ = ax.boxplot([po_vals, spo_vals], tick_labels=['PO', 'SPO'],
           showmeans=True, meanline=True)   # optional styling
ax.set_ylabel("Cost increase [%]")
ax.grid(True)

Takeaway from above: If we post-process the trained cost predictor with SPO, we can achieve a better model and better interdiction (robust) performance!

## Single Examples

In [ ]:
idx = 60

In [ ]:
# Store values for the current sample
feature = test_data["features"][idx]
cost = test_data["costs"][idx] * normalization_constant
interdiction = interdictions["costs"][idx] * normalization_constant
m, n = cfg.get("grid_size")

# Compute estimated cost if an estimator is provided. Otherwise use the true costs
po_cost = po_model(torch.tensor(feature, dtype=torch.float32)).detach().numpy() * normalization_constant
spo_cost = spo_model(torch.tensor(feature, dtype=torch.float32)).detach().numpy() * normalization_constant

# Update estimated opt_models
po_graph = ShortestPathGrid(m, n, cost=po_cost)
po_opt_model = shortestPathGrb(po_graph)

spo_graph = ShortestPathGrid(m, n, cost=spo_cost)
spo_opt_model = shortestPathGrb(spo_graph)

# Create true model
true_graph = ShortestPathGrid(m, n, cost=cost)
true_opt_model = shortestPathGrb(true_graph)

# Display estimated and true shortest paths and costs
fig, ax = plt.subplots(1, 3, figsize=(18, 6))
y_true, _ = true_opt_model.solve(visualize=True, ax=ax[0], title="True Shortest Path")
y_est, _ = po_opt_model.solve(visualize=True, ax=ax[1], title="PO Shortest Path")
y_spo, _ = spo_opt_model.solve(visualize=True, ax=ax[2], title="SPO Shortest Path")
fig.tight_layout()
print(f"True path cost: {true_graph(y_true):.3f}")
print(f"PO path cost: {true_graph(y_est):.3f}")
print(f"SPO path cost: {true_graph(y_spo):.3f}")

In [ ]:
# Solutions without information asymmetry
true_interdictor = BendersDecomposition(true_opt_model, 
                                        k=cfg.get("budget"), 
                                        interdiction_cost=interdiction, 
                                        max_cnt=cfg.get("benders_max_count"), 
                                        eps=cfg.get("benders_eps"))
x_true, y_est_true, _ = true_interdictor.solve(versatile=False)

po_interdictor = BendersDecomposition(po_opt_model, 
                                        k=cfg.get("budget"), 
                                        interdiction_cost=interdiction, 
                                        max_cnt=cfg.get("benders_max_count"), 
                                        eps=cfg.get("benders_eps"))
x_po, y_est_po, _ = po_interdictor.solve(versatile=False)

spo_interdictor = BendersDecomposition(spo_opt_model, 
                                        k=cfg.get("budget"), 
                                        interdiction_cost=interdiction, 
                                        max_cnt=cfg.get("benders_max_count"), 
                                        eps=cfg.get("benders_eps"))
x_spo, y_est_spo, _ = spo_interdictor.solve(versatile=False)

# True shortest paths after interdiction
true_opt_model.setObj(cost + x_true * interdiction)
y_true_true, _ = true_opt_model.solve()

true_opt_model.setObj(cost + x_po * interdiction)
y_true_po, _ = true_opt_model.solve()

true_opt_model.setObj(cost + x_spo * interdiction)
y_true_spo, _ = true_opt_model.solve()

# Store the results
# true_graph(y_true, interdictions=x_intd * interdiction)
# true_graph(y_est, interdictions=x_intd * interdiction)

In [ ]:
# Create figure for plotting
fig, ax = plt.subplots(2, 3, figsize=(18, 12))

# Estimated shortest paths after interdiction
true_graph.visualize(ax=ax[0, 0], colored_edges = y_est_true, dashed_edges=x_true, title="True Intd - Estimated Path")

po_graph.visualize(ax=ax[0, 1], colored_edges = y_est_po, dashed_edges=x_po, title="PO Intd - Estimated Path")

spo_graph.visualize(ax=ax[0, 2], colored_edges = y_est_spo, dashed_edges=x_spo, title="SPO Intd - Estimated Path")

# True shortest paths after interdiction
true_graph.visualize(ax=ax[1, 0], colored_edges = y_true_true, dashed_edges=x_true, title="True Intd - True Path")

po_graph.visualize(ax=ax[1, 1], colored_edges = y_true_po, dashed_edges=x_po, title="PO Intd - True Path")

spo_graph.visualize(ax=ax[1, 2], colored_edges = y_true_spo, dashed_edges=x_spo, title="SPO Intd - True Path")

fig.tight_layout()

# Print results
print(f"{true_graph(y_est_true, interdictions=x_true * interdiction)} | {true_graph(y_est_po, interdictions=x_po * interdiction)} | {true_graph(y_est_spo, interdictions=x_spo * interdiction)}")
print(f"{true_graph(y_true_true, interdictions=x_true * interdiction)} | {true_graph(y_true_po, interdictions=x_po * interdiction)} | {true_graph(y_true_spo, interdictions=x_spo * interdiction)}")

In [ ]:
# Estimated shortest paths after interdiction
true_graph.visualize(colored_edges = y_est_true, dashed_edges=x_true, title="True Intd - Estimated Path")